# Tuning Script

In [1]:
# Parameters subject to change in every script - involved in naming

MODEL = 'rf'
TUNER = 'yangzhoub'

---

In [2]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

Mounted at /content/drive


In [3]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    home_directory = './drive/MyDrive/LAB/AFLBrownlowPredictor2024/' # my home directory is stored in ./LAB of google drive
    if MODEL == 'ebr':
        !pip install interpret==0.5.0
    elif MODEL == 'cbr':
        !pip install catboost
    !pip install JXAutoML
else:
    home_directory = '../../'

from JXAutoML.YangZhouB import YangZhouB as Tuner

import pandas as pd

In [4]:
# Parameters that are same for every script
SEED = 42

TASK_TYPE = 'Regression'  # this task is a classification task

# we have decided to tune features as hp
TUNE_FEATURES_AS_HYPERPARAMETERS = True

---

In [5]:
# model to use to tune
from sklearn.ensemble import RandomForestRegressor as clf

# what values to try for each hyperparameter
parameter_choices = {
    'n_estimators': (25, 50, 100, 200, 400, 800),
    'max_depth': (3, 6, 12, 24, 48, 96, 192),
    'max_samples': (0.4, 0.55, 0.7, 0.85, 1),
    'max_features': (0.4, 0.55, 0.7, 0.85, 1),
    'ccp_alpha': (0.00000001, 0.000001, 0.0001, 0.01, 0.1)
}

# what values to set non-tuneable parameters/hyperparameters
non_tunable_hyperparameters_dict = {
    'random_state': SEED,
    'n_jobs': -1
}

---

Read in and Prepare Data

In [6]:
train_data = pd.read_csv(f'{home_directory}data/curated/modelling/train.csv')
val_data = pd.read_csv(f'{home_directory}data/curated/modelling/val.csv')
test_data = pd.read_csv(f'{home_directory}data/curated/modelling/test.csv')

train_X = train_data.drop(columns=['target'])
train_y = train_data['target']
val_X = val_data.drop(columns=['target'])
val_y = val_data['target']
test_X = test_data.drop(columns=['target'])
test_y = test_data['target']

In [7]:
# Import feature importance ordering

with open(f'{home_directory}models/feature_importance_ordering.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

Set parameters

In [8]:
# initialisation
tuner = Tuner()

print('---')

# define what model we are tuning
tuner.read_in_model(clf, TASK_TYPE, pytorch_model=False, optimised_metric='r2')

print('---')

# read in the data for training and validation
tuner.read_in_data(train_X, train_y, val_X, val_y, test_X, test_y)

print('---')

# set what hp values to tune
tuner.set_hyperparameters(parameter_choices)
# WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

# set up hp values that need to be changed from default but NOT to be tuned
tuner.set_non_tuneable_hyperparameters(non_tunable_hyperparameters_dict)

print('---')

# set up feature importance ordering

if TUNE_FEATURES_AS_HYPERPARAMETERS:
    tuner.set_features(feature_importance_ordering)
    # WARNING: this may take a while if no. tuneable hyperparameters are large

print('---')

try:  # try to read in previous results to continue tuning ...
    tuner.read_in_tuning_result_df(
        f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')
except:
    print('No previous tuning data read in')

print('---')

# set up where to save the tuning result csv
tuner.set_tuning_result_saving_address(
    f'{home_directory}models/tuning/{TUNER}_{MODEL}.csv')

print('---')

# set up where to save the current best model
tuner.set_best_model_saving_address(
    f'{home_directory}models/tmp_models/{TUNER}_{MODEL}')

YangZhouB Initialised
---
Successfully read in model <class 'sklearn.ensemble._forest.RandomForestRegressor'>, which is a Regression model optimising for r2
---
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
---
Successfully recorded hyperparameter choices
---
Successfully recorded non_tuneable_hyperparameter choices
---
Successfully recorded tuneable feature combination choices and updated relevant internal structures
---
Successfully read in tuning result of 1536 rows, for 1536.0 combos
---
Successfully set tuning output address
---
Successfully set best model output address


In [9]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# begin tuning ... (or continue training)

tuner.tune()

Streaming output truncated to the last 5000 lines.
Already Trained and Tested combination (1, 2, 0, 4, 0, 36), which had val score of 0.7536
        Current best combo: (5, 3, 3, 2, 0, 17) with val score 0.8827. 
        Has trained 1536 of 194250 combinations so far
Already Trained and Tested combination (1, 4, 0, 4, 0, 36), which had val score of 0.7903
        Current best combo: (5, 3, 3, 2, 0, 17) with val score 0.8827. 
        Has trained 1536 of 194250 combinations so far
Already Trained and Tested combination (1, 3, 1, 4, 0, 36), which had val score of 0.795
        Current best combo: (5, 3, 3, 2, 0, 17) with val score 0.8827. 
        Has trained 1536 of 194250 combinations so far
Already Trained and Tested combination (1, 3, 0, 3, 0, 36), which had val score of 0.8732
        Current best combo: (5, 3, 3, 2, 0, 17) with val score 0.8827. 
        Has trained 1536 of 194250 combinations so far
Already Trained and Tested combination (1, 3, 0, 4, 1, 36), which had val score of